In [44]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_2"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
1819  535.856285               0          0          1          0         0   
1820  546.780787               0          0          1          0         0   
1821  427.343259               0          0          0          1         0   
1822  454.859641               0          0          1          0         0   
1823  492.818804               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
3633  677.415844               0          0          0          0         1   
3634  716.576825               0          0          0          0         1   
3635  721.822447               0          0          0          1         0   
3636  616.358562               0          0          0          0         1   
3637  586.081666               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [45]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Sommer                    0.565432
is_holiday                0.402799
Monat_7                   0.395344
Sonntag                   0.385942
Monat_8                   0.371096
Temp_Warm                 0.319574
Number_of_ships           0.261063
Number_of_ships_scaled    0.261063
Ship                      0.254392
wetter_sehr_schön         0.244083
Samstag                   0.224064
Temp_Hot                  0.207087
Cloud_ok                  0.203615
Temp_Mild                 0.185650
Feiertag                  0.170925
KielerWoche               0.155228
Cloud_Clear               0.119683
Cloud_Partly_Cloudy       0.117566
Monat_6                   0.103311
Ostertag                  0.089856
Wind_Moderate             0.085326
Silvester                 0.074378
zwischen_den_jahren       0.069572
Heimspiel                 0.061620
Weather_Light_Issues      0.037564
Wind_ok                   0.027271
Markt                     0.017463
Monat_5             

In [46]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


VPI                       0.269005
Sommer                    0.200458
Werktag                   0.165373
Temp_Cold                 0.125101
Sonntag                   0.103903
Monat_7                   0.093496
is_holiday                0.091818
Winter                    0.083544
Monat_8                   0.077644
Temp_Warm                 0.060112
Temp_Mild                 0.056648
Ship                      0.053259
Number_of_ships           0.046180
Number_of_ships_scaled    0.046073
wetter_sehr_schön         0.036581
Monat_9                   0.036299
Cloud_Partly_Cloudy       0.035926
Samstag                   0.034021
Monat_1                   0.031716
Frühling                  0.030156
Cloud_ok                  0.029113
Cloud_Cloudy              0.029113
Herbst                    0.028793
Monat_10                  0.027961
Dienstag                  0.026578
Monat_6                   0.025469
Weihnachtsmarkt           0.023151
Monat_5                   0.021758
Monat_3             

In [47]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'Weihnachtsmarkt', 'Monat_12', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok']


In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold  1.169477e+08
1                Temp_Cold           inf
2                Temp_Mild           inf
3                Temp_Warm           inf
4                 Temp_Hot  1.500577e+07
5              Cloud_Clear  8.145413e+05
6      Cloud_Partly_Cloudy  1.169889e+06
7             Cloud_Cloudy  1.091062e+05
8               Wind_Light  3.870857e+04
9            Wind_Moderate  1.168463e+05
10             Wind_Strong  2.113227e+06
11            Weather_Good  1.182944e+07
12    Weather_Light_Issues  8.581310e+04
13        Weather_Moderate  3.167107e+05
14          Weather_Severe  1.810797e+07
15             KielerWoche  1.453789e+00
16                  Montag  1.906687e+06
17                Dienstag  2.154440e+06
18                Mittwoch  5.825325e+05
19              Donnerstag  1.072312e+05
20                 Freitag  6.563315e+05
21                 Samstag  1.192699e+05
22                 Sonntag  2.503423e+06
23              

In [49]:
# suche miteinander korrelierrende features


In [50]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Winter', 'Temp_Mild'), ('Ship', 'Number_of_ships'), ('is_holiday', 'Umsatz'), ('Heimspiel', 'Samstag'), ('Werktag', 'Sonntag'), ('Wind_Strong', 'Wind_Moderate'), ('Monat_11', 'Herbst'), ('Monat_7', 'Sommer'), ('Winter', 'Temp_Cold'), ('Cloud_ok', 'Cloud_Cloudy'), ('Werktag', 'Samstag'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Monat_6', 'KielerWoche'), ('Cloud_ok', 'Cloud_Clear'), ('Weather_Light_Issues', 'Weather_Good'), ('Wind_ok', 'Wind_Moderate'), ('zwischen_den_jahren', 'Silvester'), ('Monat_9', 'Herbst'), ('wetter_sehr_schön', 'Temp_Hot'), ('Monat_1', 'Winter'), ('Monat_8', 'Sommer'), ('Werktag', 'Umsatz'), ('Monat_10', 'Herbst'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Sommer', 'Temp_Cold'), ('Monat_12', 'Weihnachtsmarkt'), ('Wind_Moderate', 'Wind_Light'), ('Monat_3', 'Frühling'), ('Sommer', 'Umsatz'), ('Wind_ok', 'Wind_Strong'), ('Monat_12', 'Winter'), ('Number_of_ships_scaled', 'Ship'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Temp_C